# I. Main APACHE II Script

In [2]:
import pandas as pd
import numpy as np
import math
# Calculate APACHE II Score
# (c) 02/2018 / Arne Peine
# APACHE II ("Acute Physiology And Chronic Health Evaluation II") is a severity-of-disease classification system (Knaus et al., 1985), one of several ICU scoring systems. It is applied within 24 hours of admission of a patient to an intensive care unit (ICU): an integer score from 0 to 71 is computed based on several measurements; higher scores correspond to more severe disease and a higher risk of death. The first APACHE model was presented by Knaus et al. in 1981.
# Note: The data used should be from the initial 24 hours in the ICU, worst value should be used.
# Reference: Knaus WA Draper EA et al. APACHE II: A severity of disease classification system. Critical Care Medicine. 1985

# 1. Required Variables and explanations

# 1.1. Acute Physiology Score
# Variables are defined like this -
# Body temperature, expects float: temperature 
# Mean arterial blood pressure, expects integer: abp 
# Mean heartbeats per minute, expects integer: heart_bpm
# Respiratory rate per minute, expects integer: respiratory_rate

# Oxygenation: The APACHEII uses different inputs, depending on the fraction of inspired oxygen
# For a FiO2 > 0.5, the algorithm uses the AaDO2 (calculated like this: AaDO2 (mmHG) = pAO2 - paO2)
# The function calculate_aado2 can also be used (see below)

# For FiO2 >0.5 record AaDO2
# For FiO2 <0.5 record only PaO2 (arterial partial pressure of O2):  oxygenation 

# Blood ph level, expects float: ph 
# Blood serum sodium level in mMol/l, expects float: sodium 
# Blood serium potassium level in mMol/l, expects float: potassium 
# Acute renal injury present? 1 for yes, 0 for no, expects integer: aki
# Blood serum creatinine levels in mg/100ml, expects float: creatinine 
# Hematocrit in %, expects float: hematocrit
# White blood count (total/mm^3) in 1000s, expects float: wbc 
# Glasgow Coma Scale, expects integer: gcs 

## 2. Functions
# in_range() calculates whether a value is within the limits
# calculate_single_scores() calcuates values for a single value, e.g. calculate_single_scores (80, "abp") calculates the single score for a mean arterial blood pressure of 80
# calulate_apache2_physiology() calculates Acute Physiology Score
# chronic_health_score() calculates CHS
# calculate_apache2_final() combines all calculations above. Query like this: calculate_apache2_final(abp,temperature,heart_bpm,respiratory_rate,oxygenation,ph,sodium,potassium,hematocrit,wbc,age,scd,aki,creatinine,gcs) e.g. calculate_apache(60,37,90,20,300,7.7,190,7,50,22,44,"NO NSCD",1,7,15).

def in_range (lowerlimit, upperlimit, value, inclusive=True):
    if inclusive:
        return lowerlimit <= value <= upperlimit
    else:
        return lowerlimit <= value < upperlimit

def calculate_oxygenation(fio2, pco2, po2):
    if fio2 >= 0.50:
        oxygenation = ((fio2) * (760-47) - (pco2/0.8)) - po2
        oxygenation_cat = 'aa_gradient'
    else:
        oxygenation = po2
        oxygenation_cat = 'po2'
    return round(oxygenation, 0), oxygenation_cat

def get_params(encounter):
    abp = encounter['MAP'] 
    temperature = encounter['Temp']
    heart_bpm = encounter['HR']
    respiratory_rate = encounter['Resp']
    fio2 = encounter['FIO2, ABG']
    pco2 = encounter['PCO2, ABG']
    po2 = encounter['PO2, ABG']
    ph = encounter['PH, ABG']
    sodium = encounter['SODIUM']
    potassium = encounter['POTASSIUM']
    hematocrit = encounter['HEMATOCRIT']
    wbc = encounter['WBC']
    age = encounter['AgeAtAdmissioninYears']
    scd = encounter['Severe_Chronic_Disease']
    aki = encounter['AKI_Comorbidity']
    creatinine = encounter['CREATININE']
    gcs = encounter['gcs_total_score']
    
    return abp, temperature, heart_bpm, respiratory_rate, fio2, pco2, po2, ph, sodium, potassium, hematocrit, wbc, age, scd, aki, creatinine, gcs
def calculate_single_scores (value, selector):
    
    if selector == "abp":
        value_ranges = ([160,math.inf,4,True],[130,159,3,True],[110,129,2,True],[70,109,0,True],[50,69,2,True],[0,49,4,True])
    elif selector == "temperature":
        value_ranges = ([41,math.inf,4,True],[39,40.9,3,True],[38.5,38.9,1,True],[36,38.4,0,True],[34,35.9,1,True],[32,33.9,2,True],[30,31.9,3,True],[0,29.9,4,True])
    elif selector == "heart_bpm":
        value_ranges = ([180,math.inf,4,True],[140,179,3,True],[110,139,2,True],[70,109,0,True],[55,69,2,True],[40,54,3,True],[0,39,4,True])
    elif selector == "respiratory_rate":
        value_ranges = ([50,math.inf,4,True],[35,49,3,True],[25,34,1,True],[12,24,0,True],[10,11,1,True],[6,9,2,True],[0,5,4,True])
    elif selector == "oxygenation_aa_gradient": 
        value_ranges = ([500,math.inf,4,True],[350,499,3,True],[200,349,2,True],[0,200,0,False])
    elif selector == "oxygenation_po2": 
        value_ranges = ([71,math.inf,0,True],[61,70,1,True],[55,60,3,True],[0,55,4,False])
    elif selector == "ph":
        value_ranges = ([7.7,math.inf,4,True],[7.6,7.69,3,True],[7.5,7.59,1,True],[7.33,7.49,0,True],[7.25,7.32,2,True],[7.15,7.24,3,True],[0,7.15,4,False])
    elif selector == "sodium":
        value_ranges = ([180,math.inf,4,True],[160,179,3,True],[155,159,2,True],[150,154,1,True],[130,149,0,True],[120,129,2,True],[111,119,3,True],[0,110,4,True])
    elif selector == "potassium":
        value_ranges = ([7,math.inf,4,True],[6,6.9,3,True],[5.5,5.9,1,True],[3.5,5.4,0,True],[3,3.4,1,True],[2.5,2.9,2,True],[0,2.5,4,False])
    elif selector == "hematocrit":
        value_ranges = ([60,math.inf,4,True],[50,59.9,2,True],[46,49.9,1,True],[30,45.9,0,True],[20,29.9,2,True],[0,20,4,False])
    elif selector == "wbc":
        value_ranges = ([40,math.inf,4,True],[20,39.9,2,True],[15,19.9,1,True],[3,14.9,0,True],[1,2.9,2,True],[0,1,4,True]) 
    elif selector == "age":
        value_ranges = ([18,44,0,True],[45,54,2,True],[55,64,3,True],[65,74,5,True],[75,120,6,True]) 
    else:
        value_ranges = 0
        
    for value_range in value_ranges:
        if in_range(value_range[0], value_range[1], value, inclusive = value_range[-1]):
            return int(value_range[2])

def calculate_apache2_physiology(abp,temperature,heart_bpm,respiratory_rate,oxygenation, oxygenation_cat, ph,sodium,potassium,hematocrit,wbc,age):
    args = [val for val in list(locals().values()) if val != oxygenation_cat]
    list_of_scores = []
    if oxygenation_cat == 'aa_gradient':
        selectors = ["abp","temperature","heart_bpm","respiratory_rate","oxygenation_aa_gradient","ph","sodium","potassium","hematocrit","wbc", "age"]
    else:
        selectors = ["abp","temperature","heart_bpm","respiratory_rate","oxygenation_po2","ph","sodium","potassium","hematocrit","wbc", "age"]
    for arg, selector in zip(args, selectors):
        #print(arg, selector)
        result = calculate_single_scores(arg, selector)
        list_of_scores.append(result)
    return(list_of_scores)

def chronic_health_score(value):
    
    """
    This function accepts a total of 6 values for adressing Chronic Health Points and surgery status
    value="RO NSCD">elective postoperative, no SCD (Severe Chronic Diseases, see list)
    value="RO SCD">elective postoperative, SCD present
    value="NO NSCD">no surgery, no SCD
    value="NO SCD">no surgery, SCD present
    value="EO NSCD">emergency postoperative, no SCD
    value="EO SCD">emergency postoperative, SCD present
    
    """

    if value == "RO NSCD":
        result = "0"
    elif value == "RO SCD":
        result = "2"
    elif value == "EO NSCD":
        result = "0"
    elif value == "EO SCD":
        result = "5"
    elif value == "NO NSCD":
        result = "0"
    elif value == "NO SCD":
        result = "5"   
    return(result)
        
def calculate_apache2_final(encounter):
    """
    APACHE_score = Rectal temperature + MAP + HR + RR + A-a gradient or PaO2 + pH_or_HCO3 + Na + K + Creatinine + Hematocrit + WBC + Glasgow + Age + Chronic_diagnosis
    
    """
    
    abp, temperature, heart_bpm, respiratory_rate, fio2, pco2, po2, ph, sodium, potassium, hematocrit, wbc, age, scd, aki, creatinine, gcs = get_params(encounter)
    if(pd.isna([abp, temperature, heart_bpm, respiratory_rate, fio2, pco2, po2, ph, sodium, potassium, hematocrit, wbc, age, scd, aki, creatinine, gcs]).any()):
        return np.nan
    
    oxygenation, oxygenation_cat = calculate_oxygenation(fio2, pco2, po2)
    physiology = calculate_apache2_physiology(abp,temperature,heart_bpm,respiratory_rate,oxygenation, oxygenation_cat, 
                                              ph,sodium,potassium,hematocrit,wbc,age)
    health_points = int(chronic_health_score(scd))
    if aki == 0:  # Double points for acute kidney injury
        value_ranges = ([3.5,math.inf,4,True],[2,3.4,3,True],[1.5,1.9,2,True],[0.6,1.4,0,True],[0,0.6,2,False])
    else:
        value_ranges = ([3.5,math.inf,8,True],[2,3.4,6,True],[1.5,1.9,4,True],[0.6,1.4,0,True],[0,0.6,4,False])
    
    for value_range in value_ranges:
        if in_range(value_range[0], value_range[1], creatinine, inclusive = value_range[-1]):
            creatinine_points = int(value_range[2])
    gcs_final = 15 - gcs
    #print(physiology)
    #print(health_points)
    #print(creatinine_points)
    #print(gcs_final)
    
    try:
        return sum(physiology + [int(health_points)] + [int(creatinine_points)] + [gcs_final])
    except:
        return np.nan


# II. Load and Preprocess data before calculating APACHE II

In [3]:
pred_df = pd.read_csv('data/main_prospective_val_df_20220805_20220930_20221010.csv')
pred_df_full = pd.read_pickle('data/pred_df_full_20220805_20220930_20221010.pkl')
pred_df_full.iloc[:, 4:]

/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/machinelearning/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (228) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,DepartmentName,LocationAbbreviation,LOS,rel_time,abs_time,AdmissionInstant,AdmissionDateKey,etco2,SBP,DBP,...,SepsisLabel_0.07,SepsisLabel_0.09,SepsisLabel_0.1,SepsisLabel_0.15,SepsisLabel_0.2,SepsisLabel_0.25,SepsisLabel_0.3,SepsisLabel_0.4,SepsisLabel_0.5,Prediction_Time
0,TH 6 MAIN PERI-OP,Tisch,0.0,0.5,2022-08-03 06:36:00,2022-08-03 06:06:00,20220803,NaN,132.0,76.0,...,0,0,0,0,0,0,0,0,0,2022-11-21 18:35:13
1926313,LB 4900,LB,0.0,0.5,2022-08-24 13:42:00,2022-08-24 13:12:00,20220824,NaN,106.0,85.0,...,0,0,0,0,0,0,0,0,0,2022-11-21 18:35:13
818822,LB EMERGENCY DEPT,LB,0.0,0.5,2022-08-05 12:37:00,2022-08-05 12:07:00,20220805,NaN,156.0,98.0,...,0,0,0,0,0,0,0,0,0,2022-11-21 18:35:13
1926399,TH 14 EAST,Tisch,0.0,0.5,2022-08-24 17:19:00,2022-08-24 16:49:00,20220824,NaN,113.0,73.0,...,1,1,1,1,1,1,1,1,0,2022-11-21 18:35:13
3343443,LI NW 3 NP 3800,NYUWH,0.0,0.5,2022-09-18 08:16:00,2022-09-18 07:46:00,20220918,NaN,156.0,65.0,...,0,0,0,0,0,0,0,0,0,2022-11-21 18:35:13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352112,LB EMERGENCY DEPT,LB,1706.0,1706.5,2022-10-05 21:56:00,2022-07-26 19:26:00,20220726,NaN,115.0,79.0,...,0,0,0,0,0,0,0,0,0,2022-11-21 18:35:13
352113,LB EMERGENCY DEPT,LB,1706.5,1707.0,2022-10-05 22:26:00,2022-07-26 19:26:00,20220726,NaN,115.0,79.0,...,0,0,0,0,0,0,0,0,0,2022-11-21 18:35:13
352114,LB EMERGENCY DEPT,LB,1707.0,1707.5,2022-10-05 22:56:00,2022-07-26 19:26:00,20220726,NaN,115.0,79.0,...,0,0,0,0,0,0,0,0,0,2022-11-21 18:35:13
352115,LB EMERGENCY DEPT,LB,1707.5,1708.0,2022-10-05 23:26:00,2022-07-26 19:26:00,20220726,NaN,115.0,79.0,...,0,0,0,0,0,0,0,0,0,2022-11-21 18:35:13


In [4]:
select_cols = ['ID', 'abs_time', 'AdmissionInstant', 'rel_time', 'Temp', 'MAP',
               'HR', 'Resp','FIO2, ABG', 'PO2, ABG', 'ARTERIAL PO2, POC', 'PCO2, ABG', 'ARTERIAL PCO2, POC', 
               'PH, ABG','ARTERIAL PH, POC', 'SODIUM', 'POTASSIUM', 'CREATININE', 'HEMATOCRIT', 'WBC', 
               'gcs_total_score', 'AgeAtAdmissioninYears']
pred_df_full = pred_df_full[select_cols]
pred_df_full['PO2, ABG'] = pred_df_full['PO2, ABG'].fillna(pred_df_full['ARTERIAL PO2, POC'])
pred_df_full['PCO2, ABG'] = pred_df_full['PCO2, ABG'].fillna(pred_df_full['ARTERIAL PCO2, POC'])
pred_df_full['PH, ABG'] = pred_df_full['PH, ABG'].fillna(pred_df_full['ARTERIAL PH, POC'])

#get the first values as admission indices
pred_df_full = pred_df_full.sort_values('rel_time', ascending=True).groupby('ID', as_index=False).first(numeric_only=True)


#get comorbidities
comorb_df = pd.read_pickle('data/discharged_sepsis_pred_df_w_outcomes_matched_comorbs_20220805_20220930_20221010.pkl')
comorb_df.rename(columns={'EncounterKey':'ID'}, inplace=True)
comorb_df = comorb_df[['ID', 'Severe_Chronic_Disease','Liver_Comorbidity',
       'Cardiovascular_Comorbidity', 'Respiratory_Comorbidity',
       'Renal_Comorbidity', 'Immunocompromised_Comorbidity','AKI_Comorbidity']]
pred_df_full = pred_df_full.merge(comorb_df, on=['ID'], how='left')
pred_df_full['Severe_Chronic_Disease'] = pred_df_full['Severe_Chronic_Disease'].replace(1, 'NO SCD').replace(0, 'NO NSCD')


col_round_whole_num = ['MAP', 'HR', 'Resp', 'SODIUM']
col_round_tenths = ['Temp', 'POTASSIUM', 'WBC', 'CREATININE']
col_round_hundreths = ['PH, ABG']
col_to_floor = ['gcs_total_score', 'AgeAtAdmissioninYears']

round_dict = {col:0 for col in col_round_whole_num}
round_dict.update({col:1 for col in col_round_tenths})
round_dict.update({col:2 for col in col_round_hundreths})
pred_df_full = pred_df_full.round(round_dict)
for col in col_to_floor:
    pred_df_full[col] = pred_df_full[col].apply(np.floor)
pred_df_full['FIO2, ABG'] = pred_df_full['FIO2, ABG'].apply(lambda x: x*0.01)

In [5]:
pred_df_full.iloc[:, 4:]

,HR,Resp,"FIO2, ABG","PO2, ABG","ARTERIAL PO2, POC","PCO2, ABG","ARTERIAL PCO2, POC","PH, ABG","ARTERIAL PH, POC",SODIUM,...,WBC,gcs_total_score,AgeAtAdmissioninYears,Severe_Chronic_Disease,Liver_Comorbidity,Cardiovascular_Comorbidity,Respiratory_Comorbidity,Renal_Comorbidity,Immunocompromised_Comorbidity,AKI_Comorbidity
0,88.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.0,...,13.2,NaN,69.0,NO NSCD,0.0,0.0,0.0,0.0,0.0,0.0
1,61.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137.0,...,14.3,NaN,32.0,NO NSCD,0.0,0.0,0.0,0.0,0.0,0.0
2,79.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137.0,...,14.4,NaN,43.0,NO NSCD,0.0,0.0,0.0,0.0,0.0,0.0
3,71.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,24.6,NaN,35.0,NO NSCD,0.0,0.0,0.0,0.0,0.0,0.0
4,82.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.2,NaN,23.0,NO NSCD,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13811,108.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139.0,...,13.1,NaN,59.0,NO NSCD,0.0,0.0,0.0,0.0,0.0,0.0
13812,116.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.6,NaN,25.0,NO NSCD,0.0,0.0,0.0,0.0,0.0,0.0
13813,88.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129.0,...,16.0,NaN,85.0,NO SCD,0.0,0.0,1.0,0.0,0.0,0.0
13814,84.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.4,NaN,21.0,NO NSCD,0.0,0.0,0.0,0.0,0.0,0.0


# III. Calculate APACHE II Score

In [6]:
pred_df_full['apache_ii_score'] = pred_df_full.apply(calculate_apache2_final, axis=1)

In [8]:
pred_df_full = pred_df_full[['ID', 'Liver_Comorbidity',
       'Cardiovascular_Comorbidity', 'Respiratory_Comorbidity',
       'Renal_Comorbidity', 'Immunocompromised_Comorbidity', 'AKI_Comorbidity',
       'apache_ii_score']]
pred_df_full.iloc[:, 1:]

,Liver_Comorbidity,Cardiovascular_Comorbidity,Respiratory_Comorbidity,Renal_Comorbidity,Immunocompromised_Comorbidity,AKI_Comorbidity,apache_ii_score
0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...
13811,0.0,0.0,0.0,0.0,0.0,0.0,NaN
13812,0.0,0.0,0.0,0.0,0.0,0.0,NaN
13813,0.0,0.0,1.0,0.0,0.0,0.0,NaN
13814,0.0,0.0,0.0,0.0,0.0,0.0,NaN
